In [1]:
# Let's bring in pandas as normal
import pandas as pd

# Here’s an example. Lets create a dataframe of letter grades in descending order. We can also set an index
# value and here we'll just make it some human judgement of how good a student was, like "excellent" or "good"

df=pd.DataFrame(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D'],
                index=['excellent', 'excellent', 'excellent', 'good', 'good', 'good', 
                       'ok', 'ok', 'ok', 'poor', 'poor'],
               columns=["Grades"])
df

,Grades
excellent,A+
excellent,A
excellent,A-
good,B+
good,B
good,B-
ok,C+
ok,C
ok,C-
poor,D+


In [2]:
# Now, if we check the datatype of this column, we see that it's just an object, since we set string values
df.dtypes

Grades    object
dtype: object

In [4]:
# We can, however, tell pandas that we want to change the type to category, using the astype() function
df["Grades"].astype("category").head()

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (11, object): ['A', 'A+', 'A-', 'B', ..., 'C+', 'C-', 'D', 'D+']

现在看到由11个categories，并且pandas关注那些categories是什么。更有趣的是，这些数据不只是categorical，并且是有序的，即例如，A-在B+的前面。我们可以告诉pandas，将data根据事先创建的新的categorical data type以及在一个list中排列好的categories排序，并且令ordered = True。

In [6]:
my_categories=pd.CategoricalDtype(categories=['D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+'], 
                           ordered=True)

# then we can just pass this to the astype() function
grades=df["Grades"].astype(my_categories)
grades.head()

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (11, object): ['D' < 'D+' < 'C-' < 'C' ... 'B+' < 'A-' < 'A' < 'A+']

现在我们看到pandas不仅仅关注这里有11个categories，并且同时关注这些categories之间的顺序。这有什么用处？一个顺序的存在能帮助我们进行比较以及Boolean masking，例如，如果我们有一个成绩构成的list，然后我们将它与"C"进行比较，我们可以看到：

In [9]:
grades[grades>"C"]

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
good         B-
ok           C+
Name: Grades, dtype: category
Categories (11, object): ['D' < 'D+' < 'C-' < 'C' ... 'B+' < 'A-' < 'A' < 'A+']

有时候将categorical values分别表示为一列，以True或False表示是否满足。这在feature extraction中十分常见，并且pandas拥有一个内置函数，称作get_dummies，它将单一列的值转化为由0和1构成的多列，表示dummy variable是否存在。

现在介绍另一种scale-based operation，它将区间或比例，例如具体成绩，转化为categorical values。这听上去反直觉，因为
这种操作会丢失信息。但这在特定情况下有用，例如需要根据categories的频率可视化数据。另外，如果需要应用机器学习中的
classification，那么将使用categorical data，因此降维可能是需要的。

Pandas拥有一个函数，cut，它需要被输入一个array-like的结构，例如DataFrame的一列，或者一个Series。它同时需要被输入
所需要的"bins"的数量，每个bin的长度相同。（相当于linspace）

以census data作为例子。我们可以group by state，然后通过agg()得到一个由每个州的average county size构成的list。
如果我们还想在上面使用cut，例如10个bins，我们可以看到这些州根据average county size被列为categorical data。

In [11]:
# let's bring in numpy
import numpy as np

# Now we read in our dataset
df=pd.read_csv("C:\\Users\\asus\\Desktop\\Coursera\\Applied Data Science with Python\\(1) Introduction to Data Science in Python\\dataset\\census.csv")

# And we reduce this to country data
df=df[df['SUMLEV']==50]

# And for a few groups
df=df.set_index('STNAME').groupby(level=0)['CENSUS2010POP'].agg(np.average)

df.head()

STNAME
Alabama        71339.343284
Alaska         24490.724138
Arizona       426134.466667
Arkansas       38878.906667
California    642309.586207
Name: CENSUS2010POP, dtype: float64

In [12]:
# Now if we just want to make "bins" of each of these, we can use cut()
pd.cut(df,10)

STNAME
Alabama                   (11706.087, 75333.413]
Alaska                    (11706.087, 75333.413]
Arizona                 (390320.176, 453317.529]
Arkansas                  (11706.087, 75333.413]
California              (579312.234, 642309.586]
Colorado                 (75333.413, 138330.766]
Connecticut             (390320.176, 453317.529]
Delaware                (264325.471, 327322.823]
District of Columbia    (579312.234, 642309.586]
Florida                 (264325.471, 327322.823]
Georgia                   (11706.087, 75333.413]
Hawaii                  (264325.471, 327322.823]
Idaho                     (11706.087, 75333.413]
Illinois                 (75333.413, 138330.766]
Indiana                   (11706.087, 75333.413]
Iowa                      (11706.087, 75333.413]
Kansas                    (11706.087, 75333.413]
Kentucky                  (11706.087, 75333.413]
Louisiana                 (11706.087, 75333.413]
Maine                    (75333.413, 138330.766]
Maryland     

In [13]:
# Here we see that states like alabama and alaska fall into the same category, while california and the
# disctrict of columbia fall in a very different category.

# Now, cutting is just one way to build categories from your data, and there are many other methods. For
# instance, cut gives you interval data, where the spacing between each category is equal sized. But sometimes
# you want to form categories based on frequency – you want the number of items in each bin to the be the
# same, instead of the spacing between bins. It really depends on what the shape of your data is, and what
# you’re planning to do with it.